

## Zakres dzisiejszych zajęć:
* wdrożenie modelu ML

## Zarządzanie i wdrażanie modeli ML
Platform ML Flow (https://www.mlflow.org/docs/latest/index.html) umożliwia całościowe zarządzanie cyklem życia modeli.

Cykl życia modelu: 

1. Pobranie surowych danych  2. Przygotowanie danych 3. Trenowanie 4. Wdrożenie 

Inzynieria danych (1,2), Analityka danych (3), Inzynieria oprogramowania (4)

ML FLow ma 4 podstawowe komponenty:  TRACKING, PROJECTS, MODELS, REJESTR
* logowanie eksperymentów, wartości parametrów modeli i osiąganych przez nie wyników
* serializowanie modeli (na potrzeby współdzielenia modelu, przeniesienia na inne środowisko lub serwowania)
* samodzielny format do uruchamiania analiz
* wersjonowanie modelu, adnotowanie i przechowywanie w Rejestrze
Rozwiazanie open-source.


![mlflow](mlflow.jpeg)

Uruchomienie ML Flow: 
File >> New Launcher >> ML Flow 

(uruchamia się skrypt inicjalizujacy serwis MLFlow) 

### Przygotowanie danych do analizy

In [ ]:
import os
user_name = os.environ.get('USER')

from pyspark.sql import SparkSession
spark = SparkSession \
.builder \
.config('spark.driver.memory','1g') \
.config('spark.executor.memory', '2g') \
.getOrCreate()

gs_path = f'gs://bdg-lab-{user_name}/survey/2020/survey_results_public.csv'
db_name = user_name.replace('-','_')
spark.sql(f'DROP DATABASE IF EXISTS {db_name} CASCADE')
spark.sql(f'CREATE DATABASE {db_name}')
spark.sql(f'USE {db_name}')
table_name = "survey_2020" 

spark.sql(f'DROP TABLE IF EXISTS {table_name}')

spark.sql(f'CREATE TABLE IF NOT EXISTS {table_name} \
          USING csv \
          OPTIONS (HEADER true, INFERSCHEMA true, NULLVALUE "NA") \
          LOCATION "{gs_path}"')

spark_df= spark.sql(f'SELECT *, CAST((convertedComp > 60000) AS STRING) AS compAboveAvg \
                    FROM {table_name} WHERE convertedComp IS NOT NULL ')

### Transformacja danych do wektora cech

In [ ]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml import Pipeline
y = 'compAboveAvg' 
feature_columns = ['OpSys', 'EdLevel', 'MainBranch' , 'Country', 'JobSeek', 'YearsCode']

stringindexer_stages = [StringIndexer(inputCol=c, outputCol='strindexed_' + c).setHandleInvalid("keep") for c in feature_columns]
stringindexer_stages += [StringIndexer(inputCol=y, outputCol='label').setHandleInvalid("keep")]

onehotencoder_stages = [OneHotEncoder(inputCol='strindexed_' + c, outputCol='onehot_' + c) for c in feature_columns]
extracted_columns = ['onehot_' + c for c in feature_columns]
vectorassembler_stage = VectorAssembler(inputCols=extracted_columns, outputCol='features') 

final_columns = [y] + feature_columns + extracted_columns + ['features', 'label']

transformed_df = Pipeline(stages=stringindexer_stages + \
                          onehotencoder_stages + \
                          [vectorassembler_stage]).fit(spark_df).transform(spark_df).select(final_columns)
training, test = transformed_df.randomSplit([0.8, 0.2], seed=1234) # Podzial na zbior treningowy/testowy

###  ML Flow: Definicja eksperymentu


In [ ]:
import mlflow   ## zaimportowanie modulu 
import mlflow.spark

## tworzymy nowy eksperyment - powinien się pojawić w UI ML Flow. 
## Jesli nie tworzymy nowego eksperymentu nowe przebiegi beda sie zapisywac pod domyslnym (default)
ename = f"classifier_{user_name}"
mlflow.set_experiment(experiment_name=ename)
experiment = mlflow.get_experiment_by_name(ename)

### Definicja metryk

In [ ]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator
evaluator_auroc = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction", metricName="areaUnderROC")
evaluator_acc = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
evaluator_recall = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="weightedRecall")
evaluator_prec = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="weightedPrecision")
evaluator_f = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="weightedFMeasure")

### ML Flow: logowanie eksperymentu z drzewem decyzyjnym

Gotowy kod potoku analizy danych instrumentalizujemy z wykorzystaniem ML Flow.
Instrumentalizacja nie wplywa na proces trenowania

Wykorzystujemy zmienna srodowiskowo `MLFLOW_TRACKING_URI` ktora wskazuje gdzie dostepny jest tracking server.

In [ ]:
from pyspark.ml.classification import DecisionTreeClassifier

dt = DecisionTreeClassifier(featuresCol='features', labelCol='label')
dt_model = Pipeline(stages=[dt]).fit(training)
pred_dt = dt_model.transform(test)
label_and_pred = pred_dt.select('label', 'prediction')

### Instrumentalizacja kodu z uzyciem Pythonowego API ML Flow

with mlflow.start_run(experiment_id = experiment.experiment_id, run_name="dt_model"):
    
    mlflow.set_tag("classifier", "decision_tree")  ## ustawienie tagow
    mlflow.log_param("depth", dt.getMaxDepth())    ## zapisanie metadanych - hiperparametrow
    
    res = dt_model.transform(test)

    test_metric_auroc = evaluator_auroc.evaluate(res)
    test_metric_acc = evaluator_acc.evaluate(res)
    test_metric_recall = evaluator_recall.evaluate(res)
    test_metric_prec = evaluator_prec.evaluate(res)
    test_metric_f = evaluator_f.evaluate(res)

    mlflow.log_metric(evaluator_auroc.getMetricName(), test_metric_auroc)  ### zapisanie metryk
    mlflow.log_metric(evaluator_acc.getMetricName(), test_metric_acc) 
    mlflow.log_metric(evaluator_recall.getMetricName(), test_metric_recall) 
    mlflow.log_metric(evaluator_prec.getMetricName(), test_metric_prec)     
    mlflow.log_metric(evaluator_f.getMetricName(), test_metric_f) 
  
    mlflow.spark.log_model(spark_model=dt_model, artifact_path='dt_classifier') ## logowanie artefaktu - serializowany model

ML Flow moze zapisac model z innej zaintegrowanej biblioteki (TF, Keras etc)
Co widzimy w ML Flow UI?

### ML Flow: logowanie eksperymentu drzewa decyzyjnego z walidacją krzyżową

In [ ]:
from pyspark.ml.tuning import ParamGridBuilder
param_grid = ParamGridBuilder(). \
    addGrid(dt.maxDepth, [2,3,4,5,6]).\
    build()
from pyspark.ml.tuning import CrossValidator
cv = CrossValidator(estimator=dt, estimatorParamMaps=param_grid, evaluator=evaluator_auroc, numFolds=4)

with mlflow.start_run(experiment_id = experiment.experiment_id, run_name="best_model"):
    cv_model = cv.fit(training)
  
    mlflow.log_param("depth", cv_model.bestModel.depth)
    
    res = cv_model.bestModel.transform(test)

    test_metric_auroc = evaluator_auroc.evaluate(res)
    test_metric_acc = evaluator_acc.evaluate(res)
    test_metric_recall = evaluator_recall.evaluate(res)
    test_metric_prec = evaluator_prec.evaluate(res)
    test_metric_f = evaluator_f.evaluate(res)

    mlflow.log_metric(evaluator_auroc.getMetricName(), test_metric_auroc) 
    mlflow.log_metric(evaluator_acc.getMetricName(), test_metric_acc) 
    mlflow.log_metric(evaluator_recall.getMetricName(), test_metric_recall) 
    mlflow.log_metric(evaluator_prec.getMetricName(), test_metric_prec)     
    mlflow.log_metric(evaluator_f.getMetricName(), test_metric_f) 
  
    mlflow.spark.log_model(spark_model=cv_model.bestModel, artifact_path='best_classifier') 


### ML Flow: logowanie eksperymentu z modelem GBT
Wykorzystanie innego modelu, drzew decyzyjnych ze wzmocnieniem gradientowym.

In [ ]:
from pyspark.ml.classification import GBTClassifier
gbt = GBTClassifier(labelCol="label", featuresCol="features", maxIter=10)
gbt_model = gbt.fit(training)

with mlflow.start_run(experiment_id = experiment.experiment_id, run_name="gbt_model"):
  
    mlflow.log_param("depth", gbt.getMaxDepth())

    res = gbt_model.transform(test)
    
    test_metric_auroc = evaluator_auroc.evaluate(res)
    test_metric_acc = evaluator_acc.evaluate(res)
    test_metric_recall = evaluator_recall.evaluate(res)
    test_metric_prec = evaluator_prec.evaluate(res)
    test_metric_f = evaluator_f.evaluate(res)

    mlflow.log_metric(evaluator_auroc.getMetricName(), test_metric_auroc) 
    mlflow.log_metric(evaluator_acc.getMetricName(), test_metric_acc) 
    mlflow.log_metric(evaluator_recall.getMetricName(), test_metric_recall) 
    mlflow.log_metric(evaluator_prec.getMetricName(), test_metric_prec)     
    mlflow.log_metric(evaluator_f.getMetricName(), test_metric_f) 
  
    mlflow.spark.log_model(spark_model=gbt_model, artifact_path='gbt_classifier') 

### ML Flow: serwowanie modelu

Zeby zapewnić prosty interfejs do klasyfikatora, zapiszemy model wraz z krokami wstępnego przetwarzania (przekształcenie danych wejściowych w wektor cech)

Model jest przechowany w strukturze katalogowej - wraz z konfiguracją (zaleznosciami) oraz zserializowana reprezentacją. Moze byc przechowany w kilku "smakach" (flavour) w naszym przypadku jest to SparkML model oraz Pythonowa funkcja.

In [ ]:
import mlflow
import mlflow.spark
from pyspark.ml.classification import GBTClassifier

raw_training, raw_test = spark_df.randomSplit([0.8, 0.2], seed=1234)

full_classifier_name="full_gbt_classifier"
version=1

with mlflow.start_run(experiment_id = experiment.experiment_id, run_name="gbt_model_raw"):
    pipeline = Pipeline(stages=stringindexer_stages + \
                          onehotencoder_stages + \
                          [vectorassembler_stage] + [gbt] )
    model = pipeline.fit(raw_training)
    mlflow.spark.log_model(spark_model=model, artifact_path='gbt_classifier', registered_model_name=full_classifier_name)

### Klasyfikacja jako funkcja

In [ ]:
import mlflow.pyfunc
from pyspark.sql.functions import struct

registry_uri=f"models:/{full_classifier_name}/{version}"
pyfunc_udf = mlflow.pyfunc.spark_udf(spark, model_uri=registry_uri)

#### Przykładowe wywołanie funkcji

In [ ]:
predicted_df = raw_test.limit(10)\
    .withColumn("prediction", pyfunc_udf(struct('OpSys', 'EdLevel', 'MainBranch' , 'Country', 'JobSeek', 'YearsCode'))) \
    .select ('OpSys', 'EdLevel', 'MainBranch' , 'Country', 'JobSeek', 'YearsCode', 'prediction')
predicted_df.toPandas()

### Klasyfikacja jako serwis REST

Sekwencja komend do uruchomienia w terminalu

```bash
unset PYSPARK_SUBMIT_ARGS
source /opt/conda/etc/profile.d/conda.sh
conda activate $HOME/venv/$JUPYTER_KERNEL_NAME
mlflow models serve -m models:/full_gbt_classifier/1 -p 9090  --no-conda
```

#### Przykładowe wywołanie serwisu

In [ ]:
import requests 

url = "http://localhost:9090/invocations"
headers = {'Content-Type': 'application/json; format=pandas-split'}

input_data = '{"columns":["OpSys", "EdLevel", "MainBranch" , "Country", "JobSeek", "YearsCode"], \
    "data":[[ \
    "MacOS",\
    "Master’s degree (M.A., M.S., M.Eng., MBA, etc.)",\
    "I am a developer by profession","United Kingdom",\
    "I am not interested in new job opportunities",\
    "10"]]}'\
    .encode('utf-8')

r = requests.post(url, data=input_data, headers=headers) 
r.text



In [ ]:
spark.stop()

# Podsumowanie kursu

Podczas zajęć przećwiczyliśmy następujące technologii i narzędzi z zakresu Data&AI:
Przechowywanie:
* rozproszony system plików (HDFS)
* obiektowy system plików (GCS)

Środowiska pracy:
* lokalny klaster Hadoop 
* klaster Kubernetes w chmurze Google
* notatniki Jupyter 

Przetwarzanie danych:
* Apache Spark, API DataFrame i SQL
* Pandas
* Python

Wizualizacja danych:
* matplotlib
* seaborn



![](../img/ecosystem_green.png)